In [1]:
import afqinsight.nn.tf_models as nn
import numpy as np
import tensorflow as tf

from afqinsight.datasets import AFQDataset
from afqinsight.nn.tf_models import mlp4, cnn_lenet
from sklearn.impute import SimpleImputer

In [2]:
afq_dataset = AFQDataset(
    fn_nodes="../data/raw/combined_tract_profiles.csv",
    fn_subjects="../data/raw/participants_updated_id.csv",
    dwi_metrics=["dki_fa", "dki_md"],
    target_cols=["age"],
    index_col="subject_id",
)

In [3]:
afq_dataset.drop_target_na()

In [4]:
print(len(afq_dataset.subjects))
print(afq_dataset.X.shape)
print(afq_dataset.y.shape)

1867
(1867, 4800)
(1867,)


In [5]:
# Here we impute missing bundles.
# We impute using the entire dataset, which permits data leakage between the train and test set.
# THIS IS BAD AND SHOULDN'T BE DONE IN PRODUCTION
# But we do it here to move straight to model training
# When we are more comfortabel with the models, we should come back
# here and train the imputer only on the training set.
imputer = SimpleImputer(strategy="median")
afq_dataset.X = imputer.fit_transform(afq_dataset.X)

In [6]:
# Separate into training, test, and validation sets
dataset_size = len(afq_dataset.subjects)
train_size = int(0.7 * dataset_size)
val_size = int(0.15 * dataset_size)
test_size = int(0.15 * dataset_size)

full_dataset = afq_dataset.as_tensorflow_dataset()
full_dataset = full_dataset.shuffle(buffer_size=dataset_size, seed=0)

train_dataset = full_dataset.take(train_size)
test_dataset = full_dataset.skip(train_size)
val_dataset = test_dataset.skip(val_size)
test_dataset = test_dataset.take(test_size)

2022-01-11 19:51:03.887256: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
batch_size = 128
train_dataset = train_dataset.batch(batch_size)
test_dataset = test_dataset.batch(batch_size)
val_dataset = val_dataset.batch(batch_size)

In [8]:
# First let's check out the ``cnn_lenet`` from the tf_models module
model_cnn_lenet= cnn_lenet(input_shape=(100, 48) , n_classes=1, output_activation=None, verbose=True)

pooling layers: 4
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 48)]         0         
                                                                 
 conv1d (Conv1D)             (None, 100, 6)            870       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 50, 6)            0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 50, 16)            304       
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 25, 16)           0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 25, 26)

In [27]:
# Compile the model and fit it using training data
model_cnn_lenet.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])
model_cnn_lenet.fit(train_dataset, epochs=100, validation_data=val_dataset)

Epoch 1/100
11/11 [==============================] - 1s 41ms/step - loss: 11.0077 - mean_squared_error: 11.0077 - val_loss: 7.4126 - val_mean_squared_error: 7.4126
Epoch 2/100
11/11 [==============================] - 0s 26ms/step - loss: 11.1235 - mean_squared_error: 11.1235 - val_loss: 6.0198 - val_mean_squared_error: 6.0198
Epoch 3/100
11/11 [==============================] - 0s 26ms/step - loss: 10.2741 - mean_squared_error: 10.2741 - val_loss: 6.7171 - val_mean_squared_error: 6.7171
Epoch 4/100
11/11 [==============================] - 0s 26ms/step - loss: 10.0846 - mean_squared_error: 10.0846 - val_loss: 6.3082 - val_mean_squared_error: 6.3082
Epoch 5/100
11/11 [==============================] - 0s 26ms/step - loss: 9.6707 - mean_squared_error: 9.6707 - val_loss: 6.3514 - val_mean_squared_error: 6.3514
Epoch 6/100
11/11 [==============================] - 0s 26ms/step - loss: 9.7272 - mean_squared_error: 9.7272 - val_loss: 7.0819 - val_mean_squared_error: 7.0819
Epoch 7/100
11/11 [=

11/11 [==============================] - 0s 26ms/step - loss: 8.6187 - mean_squared_error: 8.6187 - val_loss: 5.4408 - val_mean_squared_error: 5.4408
Epoch 52/100
11/11 [==============================] - 0s 27ms/step - loss: 8.2261 - mean_squared_error: 8.2261 - val_loss: 5.6231 - val_mean_squared_error: 5.6231
Epoch 53/100
11/11 [==============================] - 0s 26ms/step - loss: 7.9730 - mean_squared_error: 7.9730 - val_loss: 6.3035 - val_mean_squared_error: 6.3035
Epoch 54/100
11/11 [==============================] - 0s 25ms/step - loss: 8.5418 - mean_squared_error: 8.5418 - val_loss: 5.5108 - val_mean_squared_error: 5.5108
Epoch 55/100
11/11 [==============================] - 0s 26ms/step - loss: 8.4324 - mean_squared_error: 8.4324 - val_loss: 5.2955 - val_mean_squared_error: 5.2955
Epoch 56/100
11/11 [==============================] - 0s 26ms/step - loss: 8.1491 - mean_squared_error: 8.1491 - val_loss: 6.1753 - val_mean_squared_error: 6.1753
Epoch 57/100
11/11 [===============

In [28]:
# Evaluate the model on the validation set
eval_cnn_lenet = model_cnn_lenet.evaluate(val_dataset)

3/3 [==============================] - 0s 5ms/step - loss: 4.3186 - mean_squared_error: 4.3186


We see that this model achieves a mean squared error of ~7.59, or an RMSE of ~2.5 years. This is probably an overly optimistic estimate because we imputed using the entire dataset.

In [29]:
# define an empty dictionary for results comparisons
results = {'cnn_lenet' : eval_cnn_lenet}

In [12]:
#``mlp4`` from the tf_models module
model_mlp4= mlp4(input_shape=(100, 48) , n_classes=1, output_activation=None, verbose=True)
# Compile the model and fit it using training data
model_mlp4.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])
model_mlp4.fit(train_dataset, epochs=100)
# Evaluate the model on the validation set
eval_mlp4 = model_mlp4.evaluate(val_dataset)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 100, 48)]         0         
                                                                 
 flatten_1 (Flatten)         (None, 4800)              0         
                                                                 
 dropout_2 (Dropout)         (None, 4800)              0         
                                                                 
 dense_3 (Dense)             (None, 500)               2400500   
                                                                 
 dropout_3 (Dropout)         (None, 500)               0         
                                                                 
 dense_4 (Dense)             (None, 500)               250500    
                                                                 
 dropout_4 (Dropout)         (None, 500)               0   

11/11 [==============================] - 0s 14ms/step - loss: 9.1025 - mean_squared_error: 9.1025
Epoch 59/100
11/11 [==============================] - 0s 13ms/step - loss: 10.2859 - mean_squared_error: 10.2859
Epoch 60/100
11/11 [==============================] - 0s 13ms/step - loss: 9.4601 - mean_squared_error: 9.4601
Epoch 61/100
11/11 [==============================] - 0s 13ms/step - loss: 8.7961 - mean_squared_error: 8.7961
Epoch 62/100
11/11 [==============================] - 0s 13ms/step - loss: 9.2761 - mean_squared_error: 9.2761
Epoch 63/100
11/11 [==============================] - 0s 14ms/step - loss: 9.2841 - mean_squared_error: 9.2841
Epoch 64/100
11/11 [==============================] - 0s 14ms/step - loss: 9.0491 - mean_squared_error: 9.0491
Epoch 65/100
11/11 [==============================] - 0s 13ms/step - loss: 10.3105 - mean_squared_error: 10.3105
Epoch 66/100
11/11 [==============================] - 0s 14ms/step - loss: 10.2692 - mean_squared_error: 10.2692
Epoch 67

In [31]:
# update the result
results.update({'mlp4': eval_mlp4})

In [14]:
from afqinsight.nn.tf_models import cnn_vgg, lstm1v0, lstm1, lstm2, blstm1, blstm2, lstm_fcn, cnn_resnet

In [32]:
#``cnn_vgg`` from the tf_models module
model_cnn_vgg= cnn_vgg(input_shape=(100, 48) , n_classes=1, output_activation=None, verbose=True)
# Compile the model and fit it using training data
model_cnn_vgg.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])
model_cnn_vgg.fit(train_dataset, epochs=100)
# Evaluate the model on the validation set
eval_cnn_vgg = model_cnn_vgg.evaluate(val_dataset)
# update the result
results.update({'cnn_vgg': eval_cnn_vgg})

LSTM(100) has changed to n_units=512; the change should impact following models: lstm1v0, lstm1, lstm2, blstm1, blstm2, lstm_fcn;

In [33]:
#``lstm1v0`` from the tf_models module
model_lstm1v0= lstm1v0(input_shape=(100, 48) , n_classes=1, output_activation=None, verbose=True)
# Compile themodel and fit it using training data
model_lstm1v0.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])
model_lstm1v0.fit(train_dataset, epochs=100)
# Evaluate the model on the validation set
eval_lstm1v0 = model_lstm1v0.evaluate(val_dataset)
# update the result
results.update({'lstm1v0': eval_lstm1v0})

In [34]:
#``lstm1`` from the tf_models module
model_lstm1= lstm1(input_shape=(100, 48) , n_classes=1, output_activation=None, verbose=True)
# Compile the model and fit it using training data
model_lstm1.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])
model_lstm1.fit(train_dataset, epochs=100, validation_data=val_dataset)
# Evaluate the model on the validation set
eval_lstm1 = model_lstm1.evaluate(val_dataset)
# update the result
results.update({'lstm1': eval_lstm1})

In [35]:
#``lstm2`` from the tf_models module
model_lstm2= lstm2(input_shape=(100, 48) , n_classes=1, output_activation=None, verbose=True)
# Compile the model and fit it using training data
model_lstm2.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])
model_lstm2.fit(train_dataset, epochs=100)
# Evaluate the model on the validation set
eval_lstm2 = model_lstm2.evaluate(val_dataset)
# update the result
results.update({'lstm2': eval_lstm2})

In [36]:
#``blstm1`` from the tf_models module
model_blstm1= blstm1(input_shape=(100, 48) , n_classes=1, output_activation=None, verbose=True)
# Compile the model and fit it using training data
model_blstm1.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])
model_blstm1.fit(train_dataset, epochs=100)
# Evaluate the model on the validation set
eval_blstm1 = model_blstm1.evaluate(val_dataset)
# update the result
results.update({'blstm1': eval_blstm1})

In [37]:
#``blstm2`` from the tf_models module
model_blstm2= blstm2(input_shape=(100, 48) , n_classes=1, output_activation=None, verbose=True)
# Compile the model and fit it using training data
model_blstm2.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])
model_blstm2.fit(train_dataset, epochs=100)
# Evaluate the model on the validation set
eval_blstm2 = model_blstm2.evaluate(val_dataset)
# update the result
results.update({'blstm2': eval_blstm2})

In [38]:
#``lstm_fcn`` from the tf_models module
model_lstm_fcn= lstm_fcn(input_shape=(100, 48) , n_classes=1, output_activation=None, verbose=True)
# Compile the model and fit it using training data
model_lstm_fcn.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])
model_lstm_fcn.fit(train_dataset, epochs=100)
# Evaluate the model on the validation set
eval_lstm_fcn = model_lstm_fcn.evaluate(val_dataset)
# update the result
results.update({'lstm_fcn': eval_lstm_fcn})

In [39]:
#``cnn_resnet`` from the tf_models module
model_cnn_resnet= cnn_resnet(input_shape=(100, 48) , n_classes=1, output_activation=None, verbose=True)
# Compile the model and fit it using training data
model_cnn_resnet.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])
model_cnn_resnet.fit(train_dataset, epochs=100)
# Evaluate the model on the validation set
eval_cnn_resnet = model_cnn_resnet.evaluate(val_dataset)
# update the result
results.update({'cnn_resnet': eval_cnn_resnet})

In [40]:
results

{'cnn_lenet': [4.318637371063232, 4.318637371063232],
 'mlp4': [10.768474578857422, 10.768474578857422],
 'cnn_vgg': [1.699886679649353, 1.699886679649353],
 'lstm1v0': [13.068290710449219, 13.068290710449219],
 'lstm1': [11.704986572265625, 11.704986572265625],
 'lstm2': [11.669118881225586, 11.669118881225586],
 'blstm1': [11.930325508117676, 11.930325508117676],
 'blstm2': [12.850606918334961, 12.850606918334961],
 'lstm_fcn': [7.87362813949585, 7.873629093170166],
 'cnn_resnet': [3.3275482654571533, 3.3275482654571533]}

Compared to conditions where n_units = 100, lstm_fcn became over-expressive when n_units = 512 and error increases.